In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sarimax/final_prev_seasons.csv


In [5]:
df = pd.read_csv("/kaggle/input/sarimax/final_prev_seasons.csv")
df

,Unnamed: 0,name,position,team,assists,bonus,bps,clean_sheets,creativity,goals_conceded,...,std transfers_in 3,std transfers_out 3,std value 3,ratio_goal_scored 3,ratio_goal_scored all,opp mean team Goal scored 3,opp mean team Goal conceded 3,opp mean team Goal scored all,opp mean team Goal conceded all,opp mean match_result all
0,0,Tim Ream,DEF,Fulham,0,0,16,0,0.8,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,David Luiz Moreira Marinho,DEF,Arsenal,0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Matteo Guendouzi,MID,Arsenal,0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Tom Cairney,MID,Fulham,0,0,11,0,10.2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Matt Macey,GK,Arsenal,0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86847,773,Shea Charles,MID,Man City,0,0,3,0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,1.666667,0.666667,1.540541,1.243243,1.513514
86848,774,Alex Robertson,MID,Man City,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,1.666667,0.666667,1.540541,1.243243,1.513514
86849,775,Nico O'Reilly,MID,Man City,0,0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,1.666667,0.666667,1.540541,1.243243,1.513514
86850,776,Yago de Santiago Alonso,MID,Spurs,0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,1.333333,2.333333,1.270270,2.000000,0.837838


In [6]:
Opt_df = df[['name', 'position', 'team', 'season', 'GW', 'value', 'total_points']]
finalDf = Opt_df[Opt_df['season']=="2022-23"].drop(columns = 'season')
finalDf.sort_values(by = 'name')
finalDf['id'] = finalDf['name'] + finalDf['team'] + finalDf['position']
finalDf

,name,position,team,GW,value,total_points,id
57234,Tim Ream,DEF,Fulham,1,45,1,Tim ReamFulhamDEF
57235,Tom Cairney,MID,Fulham,1,50,1,Tom CairneyFulhamMID
57236,Marek RodÃ¡k,GK,Fulham,1,45,1,Marek RodÃ¡kFulhamGK
57237,William Saliba,DEF,Arsenal,1,45,8,William SalibaArsenalDEF
57238,Joe Bryan,DEF,Fulham,1,45,0,Joe BryanFulhamDEF
...,...,...,...,...,...,...,...
86847,Shea Charles,MID,Man City,38,45,1,Shea CharlesMan CityMID
86848,Alex Robertson,MID,Man City,38,45,0,Alex RobertsonMan CityMID
86849,Nico O'Reilly,MID,Man City,38,45,0,Nico O'ReillyMan CityMID
86850,Yago de Santiago Alonso,MID,Spurs,38,45,0,Yago de Santiago AlonsoSpursMID


In [7]:
tempDf = finalDf[finalDf['GW'] < 11][finalDf['GW'] >= 1]
list(tempDf[tempDf['id'] == "Tim ReamFulhamDEF"].sort_values(by = "GW").reset_index()['value'])

/tmp/ipykernel_33/3208230925.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tempDf = finalDf[finalDf['GW'] < 11][finalDf['GW'] >= 1]


[45, 45, 45, 45, 45, 45, 45, 45, 45]

In [20]:
from collections import *
import math

def giveData(gw):
    defenders = []
    goalKeepers = []
    forwards = []
    midFielders = []
    data = []
    tempDf = finalDf[finalDf['GW'] < gw + 10][finalDf['GW'] >= gw]
    names = list(finalDf['id'].unique())
    for i in names:
        weights = [25,18,15,12,10,8,6,4,2,1]
        try:
            gws = list(tempDf[tempDf['id'] == i].sort_values(by = "GW").reset_index()['GW'])
            points = list(tempDf[tempDf['id'] == i].sort_values(by = "GW").reset_index()['total_points'])
            finalPoint = 0
            for idx in range(len(gws)):
                finalPoint += points[idx]*weights[gws[idx] - gw]
            finalPoint /= sum(weights)
            cost = list(tempDf[tempDf['id'] == i].sort_values(by = "GW").reset_index()['value'])[0]
            row = tempDf[tempDf['id'] == i]
            row = row.reset_index()
            finalRow = [row['name'][0],row['position'][0],row['team'][0],row['id'][0],finalPoint,cost]
            if(finalRow[1] == "DEF"):
                defenders.append(finalRow)
            elif(finalRow[1] == "MID"):
                midFielders.append(finalRow)
            elif(finalRow[1] == "FWD"):
                forwards.append(finalRow)
            else:
                goalKeepers.append(finalRow)
            
        except Exception as e:
            continue
    data.extend(goalKeepers)
    data.extend(defenders)
    data.extend(midFielders)
    data.extend(forwards)
    return data

maxDefender = 5
maxGoalKeeper = 2
maxMidFielder = 5
maxForward = 3
teamLimit = 3
budget = 1000

teams = list(finalDf['team'].unique())
print(teams)
teamsDic = defaultdict(int)
c = 0
for i in teams:
    teamsDic[i] = c
    c += 1
takenTeam = [0]*len(teams)

initData = giveData(1)
print(initData)
# squad = giveSquad(initData[0:27])
# print(squad)
# for i in range(38):
#     data = giveData(i + 1)


['Fulham', 'Arsenal', 'Aston Villa', 'West Ham', 'Southampton', 'Leicester', 'Crystal Palace', 'Spurs', "Nott'm Forest", 'Everton', 'Leeds', 'Liverpool', 'Man City', 'Newcastle', 'Bournemouth', 'Wolves', 'Chelsea', 'Brighton', 'Man Utd', 'Brentford']


/tmp/ipykernel_33/3397309119.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tempDf = finalDf[finalDf['GW'] < gw + 10][finalDf['GW'] >= gw]


[['Marek RodÃ¡k', 'GK', 'Fulham', 'Marek RodÃ¡kFulhamGK', 1.316831683168317, 45], ['Bernd Leno', 'GK', 'Fulham', 'Bernd LenoFulhamGK', 1.188118811881188, 45], ['Alphonse Areola', 'GK', 'West Ham', 'Alphonse AreolaWest HamGK', 0.24752475247524752, 45], ['Vicente Guaita', 'GK', 'Crystal Palace', 'Vicente GuaitaCrystal PalaceGK', 1.8514851485148516, 45], ['Fraser Forster', 'GK', 'Spurs', 'Fraser ForsterSpursGK', 0.0, 40], ['Wayne Hennessey', 'GK', "Nott'm Forest", "Wayne HennesseyNott'm ForestGK", 0.0, 40], ['Alex McCarthy', 'GK', 'Southampton', 'Alex McCarthySouthamptonGK', 0.0, 45], ['Alisson Ramses Becker', 'GK', 'Liverpool', 'Alisson Ramses BeckerLiverpoolGK', 2.3564356435643563, 55], ['AdriÃ¡n San Miguel del Castillo', 'GK', 'Liverpool', 'AdriÃ¡n San Miguel del CastilloLiverpoolGK', 0.0, 40], ['Illan Meslier', 'GK', 'Leeds', 'Illan MeslierLeedsGK', 2.8316831683168315, 45], ['Karl Darlow', 'GK', 'Newcastle', 'Karl DarlowNewcastleGK', 0.0, 40], ['Martin Dubravka', 'GK', 'Newcastle', 'M